In [1]:
import pandas as pd
import re
import pyspark
from pyspark.sql import SQLContext, functions, types
from pyspark.ml.feature import StringIndexer
from pyspark.sql import Row
from pyspark.conf import SparkConf
from geopy.geocoders import Nominatim
import json
import os

In [2]:
# config = SparkConf().set('spark.rpc.message.maxSize', '50')
sc=pyspark.SparkContext(appName="project")
spark = SQLContext(sc)

In [5]:
## Reading hotel dataset scraped from TripAdvisor

# h_df = pd.read_json('Testing/Project-Dataset/hotel_info.json')
# h1_df = spark.createDataFrame(h_df)
h1_df = spark.read.format('org.apache.spark.sql.json') \
        .load("Testing/Project-Dataset/hotel_info2.json").cache()
h1_df.createOrReplaceTempView('h1_df')
## Removing duplicates from the hotel dataset

temp=spark.sql("SELECT df.id FROM (SELECT id, COUNT(*) as tot_count FROM h1_df GROUP BY id ORDER BY tot_count DESC) df WHERE df.tot_count>1")
temp.createOrReplaceTempView('temp')
del_dup = spark.sql("SELECT h1_df.* FROM h1_df LEFT JOIN temp ON h1_df.id == temp.id WHERE temp.id IS NULL").cache()
del_dup.createOrReplaceTempView('del_dup')

del_dup.write.mode("overwrite").json("Testing/Project-Dataset/del_dup/city1.json")

In [14]:
## Getting city using coordinates

geolocator = Nominatim(user_agent="new_recomm", timeout=None)

# def get_cname(x):
#     if 'nil' in x[1:-1]:
#         return "None"
#     else:
#         location = geolocator.reverse(x[1:-1], timeout=None)
#         if 'city' in location.raw["address"]:
#             return location.raw["address"]["city"]
#         elif 'town' in location.raw["address"]:
#             return location.raw["address"]["town"]
#         else:
#             return "None"
    
# get_city = functions.udf(lambda a:get_cname(a),types.StringType())

city_df = del_dup.withColumn("location", del_dup.location)

## Saving etled dataset

city_df.createOrReplaceTempView('del_dup')
city_df.write.mode("overwrite").json("Testing/Project-Dataset/del_dup")
# city_df.write.json('Testing/Project-Dataset/del_dup')
# with open("Testing/Project-Dataset/del_dup/city1.json", "w+") as output_file:
#     output_file.write(city_df.toJSON())

In [15]:
## Explode amenities to make predictions based on length of amentities provided by user

newh_df  = spark.sql("SELECT id,explode(amenities) as amenities FROM del_dup")

##  Removing punctuations from amenities column

strip_udf = functions.udf(lambda x: re.sub(r'[^\w\s]','',x), types.StringType())
newh_df = newh_df.withColumn("amenities", strip_udf(functions.col("amenities")))
newh_df.createOrReplaceTempView('newh_df')
# newh_df.coalesce(4).write.json('Testing/Project-Dataset/newh_df',mode='overwrite')

AnalysisException: cannot resolve 'explode(del_dup.`amenities`)' due to data type mismatch: input to function explode should be array or map type, not struct<0:array<string>,1:array<string>,10:array<string>,11:array<string>,12:array<string>,13:array<string>,14:array<string>,15:array<string>,16:array<string>,17:array<string>,18:array<string>,19:array<string>,2:array<string>,20:array<string>,21:array<string>,22:array<string>,23:array<string>,24:array<string>,25:array<string>,26:array<string>,27:array<string>,28:array<string>,29:array<string>,3:array<string>,30:array<string>,31:array<string>,32:array<string>,33:array<string>,34:array<string>,35:array<string>,36:array<string>,37:array<string>,38:array<string>,39:array<string>,4:array<string>,40:array<string>,41:array<string>,42:array<string>,43:array<string>,44:array<string>,45:array<string>,46:array<string>,47:array<string>,48:array<string>,49:array<string>,5:array<string>,6:array<string>,7:array<string>,8:array<string>,9:array<string>>; 

In [ ]:
df = pd.read_json('tripadvisor_hotel_output/reviews.json')

df["att_id"]=df.id.astype('category').cat.codes

rev_df = spark.createDataFrame(df).cache()
rev_df.createOrReplaceTempView('rev_df')

rev_temp=spark.sql("SELECT df.id FROM (SELECT id, COUNT(*) as tot_count FROM rev_df GROUP BY id ORDER BY tot_count DESC) df WHERE df.tot_count>1")
rev_temp.createOrReplaceTempView('rev_temp')

s_df = spark.sql("SELECT rev_df.* FROM rev_df LEFT JOIN rev_temp ON rev_df.id == rev_temp.id WHERE rev_temp.id IS NULL")
s_df.createOrReplaceTempView('s_df')


## String Indexing user_name 

indexer = StringIndexer(inputCol="user_name", outputCol="user_id")
indexed = indexer.fit(s_df).transform(s_df)
u_id_df = indexed.withColumn("user_id",indexed["user_id"].cast("Int")).cache()
u_id_df.createOrReplaceTempView('u_id_df')
u_id_df.coalesce(4).write.json('etl/u_id_df',mode='overwrite')